In [6]:
import numpy as np
import pandas as pd
import pickle
import os
import gdown
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.callbacks import ModelCheckpoint
import joblib
import time

# Import Models and Datasets

In [7]:
# Should take about 3 minutes if files not already present
# Import pretrained models
path = "./"
m = "model_"
models = ["lr.sav", "rf.sav", "xgb.sav", "ffnn.keras", "cnn.keras"]
model_urls = [
    "https://drive.google.com/file/d/1cQfOEH3b6-dxVMkEL2aM8N5p5Zt9jNJw/view?usp=drive_link",
    "https://drive.google.com/file/d/1OqREa20mOlTnZVdsZ-_lKBfkwIl4fJYG/view?usp=drive_link",
    "https://drive.google.com/file/d/1Q4mzsXIL4bX8sXpG6mMpvjCmvXgT9eh2/view?usp=drive_link",
    "https://drive.google.com/file/d/1AV2IFpEU7EGn4ZOJWgmKdoq-ghxUaEjD/view?usp=drive_link",
    "https://drive.google.com/file/d/1Try-1FfpfbXPJGq94wm9ZuawaZHZIzuU/view?usp=drive_link",
]

# Import Incompleted studies to add to Dashboard DataFrame
x = "X_"
y = "y_"
incompleted = "incompleted.pkl"
groups = ["X_incompleted.pkl", "X_incompleted_pca.pkl", "y_incompleted.pkl"]
group_urls = [
    "https://drive.google.com/file/d/1-EmGbMENBqMZwH9xxyNujwKnsDiVL3mD/view?usp=drive_link",
    "https://drive.google.com/file/d/1RQgpE9CgNhpJAKqIX5z_wi3yItRu7pCo/view?usp=drive_link",
    "https://drive.google.com/file/d/1Z3KF8b1ntL5quFkv71aGD0vF6DNM8S4Q/view?usp=drive_link",
]

for i, model in enumerate(models):
    if os.path.isfile(path + m + model):
        pass
    else:
        output = m + model

        gdown.download(model_urls[i], output, fuzzy=True)


for i, group in enumerate(groups):
    if os.path.isfile(path + group):
        pass
    else:
        output = group

        gdown.download(group_urls[i], output, fuzzy=True)

Downloading...
From: https://drive.google.com/uc?id=1cQfOEH3b6-dxVMkEL2aM8N5p5Zt9jNJw
To: c:\Users\97dan\TrialDuraPredict\src\model_lr.sav
100%|██████████| 43.6k/43.6k [00:00<00:00, 792kB/s]
Downloading...
From: https://drive.google.com/uc?id=1OqREa20mOlTnZVdsZ-_lKBfkwIl4fJYG
To: c:\Users\97dan\TrialDuraPredict\src\model_rf.sav
100%|██████████| 94.7M/94.7M [00:03<00:00, 24.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1Q4mzsXIL4bX8sXpG6mMpvjCmvXgT9eh2
To: c:\Users\97dan\TrialDuraPredict\src\model_xgb.sav
100%|██████████| 2.39M/2.39M [00:00<00:00, 9.59MB/s]
Downloading...
From: https://drive.google.com/uc?id=1AV2IFpEU7EGn4ZOJWgmKdoq-ghxUaEjD
To: c:\Users\97dan\TrialDuraPredict\src\model_ffnn.keras
100%|██████████| 4.18M/4.18M [00:00<00:00, 13.0MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1Try-1FfpfbXPJGq94wm9ZuawaZHZIzuU
From (redirected): https://drive.google.com/uc?id=1Try-1FfpfbXPJGq94wm9ZuawaZHZIzuU&confirm=t&uuid=e3cbeeb2-fd0f-4fd2-a2dc-baf0980e

In [8]:
# import Dashboard DataFrame
if os.path.isfile("./study_info_df.csv"):
    pass
else:
    # Should only take ~30 seconds
    url = "https://drive.google.com/file/d/1sSYhHvWkYVRFoB1ulEblmTwo9iSibYpG/view?usp=drive_link"

    output = "study_info_df.csv"

    gdown.download(url, output, fuzzy=True)

study_info_df = pd.read_csv("./study_info_df.csv")
study_info_df

,nctId,status,phases,description,inclusion_criteria,exclusion_criteria,intervention_type,intervention_name,disease,outcome_measures,outcome_timeframes,durationMonths
0,NCT03202394,COMPLETED,['PHASE2'],"This is a randomized, double blind, placebo co...",Inclusion Criteria:\n\n1. Has provided (or rel...,:\n\n1. Age \< 18 years or \>75 years old\n2. ...,"['DRUG', 'DRUG']","['BIO-11006', 'Placebo']","['Respiratory Distress Syndrome, Adult']",['Incidence of treatment-emergent adverse even...,['28 days'],32.266667
1,NCT02138214,COMPLETED,['PHASE2'],PRIMARY OBJECTIVES:\n\nI. To determine the rat...,Inclusion Criteria:\n\n* Pre-operative diagnos...,:\n\n* Largest papillary thyroid carcinoma \< ...,"['PROCEDURE', 'PROCEDURE', 'OTHER']","['Thyroidectomy', 'entral lymph node dissectio...","['Stage I Papillary Thyroid Cancer', 'Stage II...",['Percentage of Participants With Transient Hy...,"['Post-operative day 1', 'At day 12', '2 weeks...",65.733333
2,NCT00976274,COMPLETED,['NA'],The details of study objective are followed by...,Inclusion Criteria:(three or more of following...,: (any one of following factors)\n\n* uncontro...,"['DIETARY_SUPPLEMENT', 'DIETARY_SUPPLEMENT']","['Korean red ginseng', 'starch']",['Metabolic Syndrome'],['Change in the Pre- and Post-treatment Systol...,['baseline and 12 weeks'],17.266667
3,NCT05529719,COMPLETED,['NA'],Objective: This study evaluates the effect of ...,Inclusion Criteria:\n\n* 18 to 30 years of age...,:\n\n* Volunteers who did not meet the inclusi...,['OTHER'],['Clinical Pilates Exercises'],['Low Back Pain'],['Prim Outcome - Oswestry Dysability Index'],['12 weeks'],25.266667
4,NCT02323191,COMPLETED,['PHASE1'],"This Phase 1, open-label, multicenter, global ...",Inclusion Criteria:\n\n* Eastern Cooperative O...,\n* Measurable disease at baseline as per RECI...,"['DRUG', 'DRUG']","['Atezolizumab', 'Emactuzumab']",['Solid Cancers'],['Percentage of Participants With Dose Limitin...,"['21 days', '21 days', 'Baseline up to 3 years']",68.033333
...,...,...,...,...,...,...,...,...,...,...,...,...
184620,NCT05883852,RECRUITING,['PHASE3'],The objective of this study is to conduct a ra...,Inclusion Criteria:\n\n* Women aged 18-70；\n* ...,:\n\n* Bilateral breast cancer or carcinoma in...,"['DRUG', 'DRUG', 'DRUG', 'DRUG', 'DRUG', 'DRUG']","['Docetaxel', 'carboplatin', 'Trastuzumab', 'P...",['HER2 Positive Early Breast Cancer'],['iDFS'],['5 years'],98.200000
184621,NCT02180724,ACTIVE_NOT_RECRUITING,['PHASE2'],Clinical studies have shown that targeting the...,Inclusion Criteria:\n\n1. Men and women ≥18 ye...,":\n\n1. Prior malignancy, except for adequatel...","['DRUG', 'DRUG']","['Acalabrutinib (ACP-196)', 'Acalabrutinib (AC...",['Waldenström Macroglobulinemia (WM)'],['Overall Response Rate (ORR) of Acalabrutinib...,['Up to approximately 3.8 years. Data cut at w...,61.633333
184622,NCT00487305,ACTIVE_NOT_RECRUITING,['PHASE1'],* The dose of vaccine will depend upon how man...,Inclusion Criteria:\n\n* Histologically confir...,:\n\n* Uncontrolled active infection or illnes...,['BIOLOGICAL'],['Lethally Irradiated Lymphoma cells with GM-C...,['Follicular Lymphoma'],['To determine the safety and toxicity of admi...,['2 years'],195.800000
184623,NCT05786924,RECRUITING,['PHASE1'],"BDTX-4933-101 is a first-in-human, open-label,...",Key Inclusion Criteria:\n\n1. Disease criteria...,:\n\n1. Cancer that has a known MEK1/2 mutatio...,['DRUG'],['BDTX-4933'],"['Non-small Cell Lung Cancer', 'Histiocytic Ne...",['Dose Escalation: Incidence of dose-limiting ...,"['The first 28-day cycle (Cycle 1)', 'Day 1 of...",38.000000


In [9]:
# Test
# with open("../_test/X_test.pkl", "rb") as file:
#     X_test = pickle.load(file)

# with open("../_test/y_test.pkl", "rb") as file:
#     y_test = pickle.load(file)

# Incomplete
with open("./X_incompleted.pkl", "rb") as file:
    X_incompleted = pickle.load(file)

with open("./y_incompleted.pkl", "rb") as file:
    y_incompleted = pickle.load(file)

# PCA versions
# with open("../_test/X_train_pca.pkl", "rb") as file:
#     X_train_pca = pickle.load(file)

with open("./X_incompleted_pca.pkl", "rb") as file:
    X_incompleted_pca = pickle.load(file)

# Linear Regression

In [10]:
# Load in pretrained model and get predicted values
lr = joblib.load("../_test/model_lr.sav")
y_pred_lr = lr.predict(X_incompleted)
y_pred_lr

c:\Users\97dan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Ridge from version 1.5.1 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


array([19.69473811, 19.36546213,  5.28051592, ..., 71.65103307,
       54.29086775, 26.15483794])

# Random Forest

In [11]:
# Load in pretrained model and get predicted values
rf = joblib.load("../_test/model_rf.sav")
y_pred_rf = rf.predict(X_incompleted_pca)
y_pred_rf

c:\Users\97dan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.5.1 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\97dan\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.5.1 when using version 1.3.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


array([40.26951708, 15.44139709, 24.77514241, ..., 63.95204551,
       52.45273405, 21.9216853 ])

# XGBoost

In [12]:
# Load in pretrained model and get predicted values
with open("../_test/model_xgb.sav", "rb") as f:
    xg = pickle.load(f)

y_pred_xgb = xg.predict(X_incompleted_pca)
y_pred_xgb

c:\Users\97dan\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [01:36:57] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


array([41.311485, 16.102915, 22.86373 , ..., 69.25941 , 54.54035 ,
       15.755807], dtype=float32)

# FFNN

In [13]:
# Load in pretrained model and get predicted values
ffnn = load_model(
    "../_test/model_ffnn.keras", custom_objects=None, compile=True, safe_mode=True
)
y_pred_ffnn = ffnn.predict(X_incompleted)
y_pred_ffnn

1612/1612 ━━━━━━━━━━━━━━━━━━━━ 1s 649us/step


array([[25.756376],
       [13.53969 ],
       [17.498016],
       ...,
       [57.744637],
       [50.911907],
       [26.661888]], dtype=float32)

# CNN

In [14]:
# Load in pretrained model and get predicted values
cnn = load_model(
    "../_test/model_cnn.keras", custom_objects=None, compile=True, safe_mode=True
)
y_pred_cnn = cnn.predict(X_incompleted)
y_pred_cnn

1612/1612 ━━━━━━━━━━━━━━━━━━━━ 57s 35ms/step


array([[27.747694],
       [13.165511],
       [19.505198],
       ...,
       [61.075123],
       [50.79863 ],
       [22.99662 ]], dtype=float32)

# Combine All Model Predictions

In [15]:
# Compare the mse to see which predictions to use
preds = [y_pred_lr, y_pred_rf, y_pred_xgb, y_pred_ffnn, y_pred_cnn]
models = ["Linear Regression", "Random Forest", "XGBoost", "FFNN", "CNN"]

for i, p in enumerate(preds):
    try:
        mse = mean_squared_error(y_incompleted, p)
    except:
        mse = mean_squared_error(y_test, p)

    print(models[i], mse)
    # Around 29 error

Linear Regression 834.8014383046134
Random Forest 885.2747287659092
XGBoost 821.7040187854658
FFNN 821.5441708303307
CNN 823.3580559978944


# Add best model predictions to Dashboard DataFrame

In [16]:
# DELETE ONCE WHOLE NOTEBOOK WORKS
# study_info_df = pd.read_csv("./study_info_df.csv")

study_info_df

,nctId,status,phases,description,inclusion_criteria,exclusion_criteria,intervention_type,intervention_name,disease,outcome_measures,outcome_timeframes,durationMonths
0,NCT03202394,COMPLETED,['PHASE2'],"This is a randomized, double blind, placebo co...",Inclusion Criteria:\n\n1. Has provided (or rel...,:\n\n1. Age \< 18 years or \>75 years old\n2. ...,"['DRUG', 'DRUG']","['BIO-11006', 'Placebo']","['Respiratory Distress Syndrome, Adult']",['Incidence of treatment-emergent adverse even...,['28 days'],32.266667
1,NCT02138214,COMPLETED,['PHASE2'],PRIMARY OBJECTIVES:\n\nI. To determine the rat...,Inclusion Criteria:\n\n* Pre-operative diagnos...,:\n\n* Largest papillary thyroid carcinoma \< ...,"['PROCEDURE', 'PROCEDURE', 'OTHER']","['Thyroidectomy', 'entral lymph node dissectio...","['Stage I Papillary Thyroid Cancer', 'Stage II...",['Percentage of Participants With Transient Hy...,"['Post-operative day 1', 'At day 12', '2 weeks...",65.733333
2,NCT00976274,COMPLETED,['NA'],The details of study objective are followed by...,Inclusion Criteria:(three or more of following...,: (any one of following factors)\n\n* uncontro...,"['DIETARY_SUPPLEMENT', 'DIETARY_SUPPLEMENT']","['Korean red ginseng', 'starch']",['Metabolic Syndrome'],['Change in the Pre- and Post-treatment Systol...,['baseline and 12 weeks'],17.266667
3,NCT05529719,COMPLETED,['NA'],Objective: This study evaluates the effect of ...,Inclusion Criteria:\n\n* 18 to 30 years of age...,:\n\n* Volunteers who did not meet the inclusi...,['OTHER'],['Clinical Pilates Exercises'],['Low Back Pain'],['Prim Outcome - Oswestry Dysability Index'],['12 weeks'],25.266667
4,NCT02323191,COMPLETED,['PHASE1'],"This Phase 1, open-label, multicenter, global ...",Inclusion Criteria:\n\n* Eastern Cooperative O...,\n* Measurable disease at baseline as per RECI...,"['DRUG', 'DRUG']","['Atezolizumab', 'Emactuzumab']",['Solid Cancers'],['Percentage of Participants With Dose Limitin...,"['21 days', '21 days', 'Baseline up to 3 years']",68.033333
...,...,...,...,...,...,...,...,...,...,...,...,...
184620,NCT05883852,RECRUITING,['PHASE3'],The objective of this study is to conduct a ra...,Inclusion Criteria:\n\n* Women aged 18-70；\n* ...,:\n\n* Bilateral breast cancer or carcinoma in...,"['DRUG', 'DRUG', 'DRUG', 'DRUG', 'DRUG', 'DRUG']","['Docetaxel', 'carboplatin', 'Trastuzumab', 'P...",['HER2 Positive Early Breast Cancer'],['iDFS'],['5 years'],98.200000
184621,NCT02180724,ACTIVE_NOT_RECRUITING,['PHASE2'],Clinical studies have shown that targeting the...,Inclusion Criteria:\n\n1. Men and women ≥18 ye...,":\n\n1. Prior malignancy, except for adequatel...","['DRUG', 'DRUG']","['Acalabrutinib (ACP-196)', 'Acalabrutinib (AC...",['Waldenström Macroglobulinemia (WM)'],['Overall Response Rate (ORR) of Acalabrutinib...,['Up to approximately 3.8 years. Data cut at w...,61.633333
184622,NCT00487305,ACTIVE_NOT_RECRUITING,['PHASE1'],* The dose of vaccine will depend upon how man...,Inclusion Criteria:\n\n* Histologically confir...,:\n\n* Uncontrolled active infection or illnes...,['BIOLOGICAL'],['Lethally Irradiated Lymphoma cells with GM-C...,['Follicular Lymphoma'],['To determine the safety and toxicity of admi...,['2 years'],195.800000
184623,NCT05786924,RECRUITING,['PHASE1'],"BDTX-4933-101 is a first-in-human, open-label,...",Key Inclusion Criteria:\n\n1. Disease criteria...,:\n\n1. Cancer that has a known MEK1/2 mutatio...,['DRUG'],['BDTX-4933'],"['Non-small Cell Lung Cancer', 'Histiocytic Ne...",['Dose Escalation: Incidence of dose-limiting ...,"['The first 28-day cycle (Cycle 1)', 'Day 1 of...",38.000000


In [17]:
# Get incompleted NCTIDs to use to merge with Dashboard DataFrame
incompleted_ids = pd.read_csv("../results/incompleted_ids.csv")
incompleted_ids

,nctId,status,studyType,primaryPurpose
0,NCT05499390,ACTIVE_NOT_RECRUITING,INTERVENTIONAL,TREATMENT
1,NCT05441683,RECRUITING,INTERVENTIONAL,TREATMENT
2,NCT05613686,RECRUITING,INTERVENTIONAL,TREATMENT
3,NCT05497843,RECRUITING,INTERVENTIONAL,TREATMENT
4,NCT06157541,RECRUITING,INTERVENTIONAL,TREATMENT
...,...,...,...,...
52164,NCT05883852,RECRUITING,INTERVENTIONAL,TREATMENT
52165,NCT02180724,ACTIVE_NOT_RECRUITING,INTERVENTIONAL,TREATMENT
52166,NCT00487305,ACTIVE_NOT_RECRUITING,INTERVENTIONAL,TREATMENT
52167,NCT05786924,RECRUITING,INTERVENTIONAL,TREATMENT


In [18]:
# Add predicted durations onto df, removing ones that were unable to be embedded due to NaNs
incompleted_preds_df = study_info_df[
    study_info_df.nctId.isin(list(incompleted_ids.nctId))
].dropna(
    subset=[
        "intervention_name",
        "exclusion_criteria",
        "outcome_measures",
        "outcome_timeframes",
    ]
)
incompleted_preds_df["predicted_durationMonths"] = y_pred_ffnn
incompleted_preds_df = incompleted_preds_df[
    ["nctId", "predicted_durationMonths"]
].reset_index(drop=True)
incompleted_preds_df

,nctId,predicted_durationMonths
0,NCT05499390,25.756376
1,NCT05441683,13.539690
2,NCT05613686,17.498016
3,NCT05497843,38.387852
4,NCT06157541,42.911953
...,...,...
51548,NCT05883852,48.514359
51549,NCT02180724,25.597456
51550,NCT00487305,57.744637
51551,NCT05786924,50.911907


In [85]:
# Save for use with TrialDuraPredict_Dashboard Jupyter Notebook file
incompleted_preds_df.to_csv("../results/incompleted_preds_df.csv", index=False)